In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras

import numpy as np
import os
import time

In [32]:
df = tfds.load('ag_news_subset', split='train', shuffle_files=True, download=False)
text=tfds.as_dataframe(df.take(10000))['title'].to_string()

In [33]:
vocab = ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [34]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [35]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[49, 50, 51, 52, 53, 54, 55], [72, 73, 74]]>

In [36]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [37]:
tf.strings.reduce_join(chars, axis=-1).numpy()
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [38]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
seq_length = 100

In [39]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

tf.Tensor(
[b'0' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b'b' b"'" b'A' b'M' b'D' b' ' b'D' b'e' b'b' b'u' b't' b's'
 b' ' b'D' b'u' b'a' b'l' b'-' b'C' b'o' b'r' b'e' b' ' b'O' b'p' b't'
 b'e' b'r' b'o' b'n' b' ' b'P' b'r' b'o' b'c' b'e' b's' b's' b'o' b'r'
 b"'" b'\n' b'1' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'b' b'[UNK]' b'W' b'o' b'o' b'd'
 b"'" b's' b' ' b'S' b'u' b's' b'p' b'e' b'n' b's' b'i' b'o' b'n' b' '
 b'U' b'p' b'h'], shape=(101,), dtype=string)
b"0               b'AMD Debuts Dual-Core Opteron Processor'\n1                   b[UNK]Wood's Suspension Uph"
b"eld [UNK]Reuters[UNK][UNK]\n2          b'Bush reform may have blue states seeing red'\n3                    b[UNK]'Halt"
b" science decline in schools'[UNK]\n4                              b'Gerrard leaves practice'\n5            "
b"b'Sony Banking on MGM Deal to Boost Profits'\n6              b'Giant pandas in China reserve ge

In [40]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [41]:
split_input_target(list("Tensorflow"))
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"0               b'AMD Debuts Dual-Core Opteron Processor'\n1                   b[UNK]Wood's Suspension Up"
Target: b"               b'AMD Debuts Dual-Core Opteron Processor'\n1                   b[UNK]Wood's Suspension Uph"


In [42]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [43]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [44]:
@keras.saving.register_keras_serializable(package='my_package')
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units, **kwargs):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.rnn_units = rnn_units

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(tf.shape(x)[0])
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x
  
  def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "embedding_dim": self.embedding_dim,
            "rnn_units": self.rnn_units,
        })
        return config

  @classmethod
  def from_config(cls,config):
    return cls(**config)

In [45]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [46]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch)
    example_batch_predictions = model(np.array(input_example_batch))
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
model.summary()

tf.Tensor(
[[53 49 60 ... 67  2 71]
 [66 73  2 ... 57 60 57]
 [ 2  2  2 ...  2  2  2]
 ...
 [50 60 63 ...  0  0  4]
 [60  2 67 ...  2  2  2]
 [53 67  2 ... 56 49 67]], shape=(64, 100), dtype=int64)
(64, 100, 75) # (batch_size, sequence_length, vocab_size)


Model: "my_model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (64, 100, 256)         │        19,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ ((64, 100, 1024), (64, │     3,938,304 │
│                                 │ 1024))                 │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (64, 100, 75)          │        76,875 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,034,379 (15.39 MB)

 Trainable params: 4,034,379 (15.39 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([72,  4, 59, 45,  2, 39, 41, 59,  9, 31, 41, 68, 25,  9, 37, 19, 74,
       41, 62, 57, 21, 71, 71, 46, 43, 32, 49, 38, 34, 74, 46, 34, 24, 68,
       31, 25, 64, 15,  5, 40, 23, 59, 37, 40, 13, 49, 43, 32,  2, 69, 71,
       37, 55, 19, 37, 11, 42, 62, 10, 26,  2, 53, 26, 73, 23, 27, 74, 59,
       63, 21, 30, 73, 64, 70,  2, 73, 50, 62,  0, 34, 69, 43, 63, 65, 30,
       18, 24, 67, 58,  7, 16, 40, 27, 33, 66, 51, 46, 20, 63, 10],
      dtype=int64)

In [48]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"eal'\n472                   b'Video phones act as dating tools'\n473                          b'This w"

Next Char Predictions:
 b'x$kW QSk.IStC.O9zSni;wwXUJaPLzXLBtICp5&RAkOR3aUJ uwOg9O1Tn0D eDyAEzko;Hypv ybn[UNK]LuUoqH8Bsj,6REKrcX:o0'


In [49]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()
model.compile(optimizer='adam', loss=loss)

Prediction shape:  (64, 100, 75)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.3158183, shape=(), dtype=float32)


In [50]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.keras")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix)

In [ ]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/3
89/89 ━━━━━━━━━━━━━━━━━━━━ 405s 4s/step - loss: 2.9318
Epoch 2/3
89/89 ━━━━━━━━━━━━━━━━━━━━ 291s 3s/step - loss: 1.9562
Epoch 3/3
89/89 ━━━━━━━━━━━━━━━━━━━━ 291s 3s/step - loss: 1.8109


In [54]:
#model.save('News.keras')
custom_objects={'my_package':MyModel}
with keras.saving.custom_object_scope(custom_objects=custom_objects):
  model = tf.keras.models.load_model('News.keras',custom_objects=custom_objects)

In [55]:
@keras.saving.register_keras_serializable(package='my_package2')
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0, **kwargs):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

  def get_config(self):
        config = super().get_config()
        config.update({
            "model": self.model,
            "chars_from_ids": self.chars_from_ids,
            "ids_from_chars": self.ids_from_chars,
            "temperature": self.temperature,
        })
        return config

  @classmethod
  def from_config(cls, config):
        model = keras.saving.deserialize_keras_object(config.pop("model"))
        chars_from_ids = keras.saving.deserialize_keras_object(config.pop("chars_from_ids"))
        ids_from_chars = keras.saving.deserialize_keras_object(config.pop("ids_from_chars"))
        return cls(model=model,
                   chars_from_ids=chars_from_ids,
                   ids_from_chars=ids_from_chars,
                   **config)

In [56]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [57]:
start = time.time()
states = None
next_char = tf.constant(['Breaking:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Breaking:  rivald'
9800             b'Wormment to Masen Ruthink palligyid'
9396      b'END Cotress Bring interive tramon !heroky Dill Fore Pebuld'
8096    b'Atmrepbati Worries HMO Flard Daping Hacn: Ch...
2808    b'Jupst: Ite near Nexpen Hudea-Qwuel 19090  bma...
9788        b'Thed Alpantining Acrains Eets Stry Shiant'
9000    b'Al-Mow-2 Gows C-Reputs to Ferms Telion Carne NAP'
6762                     b'Opeca tarkel Hail Minge'
6031           b'Themmates Tord her agonved acreaded'
8533              b'Ning-Chart: Fele Couthto brofind '3904...
7441       b'Itto Contrough to Revor T- Clage Studs no Furri...
7882                            b'Sporks hote tree Toll'
9281    b'UE Bun Inter Toll-in pendictres Revail Priffe...
9977              b'Ind Couble Polmbers Nibuter Enghan'
9208    b'Signof wamns 1- Ryofger Plana Ats Oullys'
9674          b'Suutch Off Fitinel Cap Oud The FB'
9804                     b'Fy Inderst 3- JuyDr: Rovitiat'
5912          b'Stowy nele ward shream fives in Walis'

In [58]:
start = time.time()
states = None
next_char = tf.constant(['Breaking:', 'Sports:', 'Technology:', 'Politics:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"Breaking:\n7855         b'Atchebs serke tovedmires mosoft ain OG..\n9512                                      b'US Pellian Chunce Sear'\n9385    b'AP to AC now shurmutixht offerity Wunte...\n4746                b'Lay Extle Fill-ked Toll 2- -GAO Op'\n8991          b'Sonchede waln soffout sundurigntepters quic...\n9000                b'Hustory Nex Wound Voorts stops insicust'\n9780                b'EDM State wivit up Yorse rope'\n9462    b'Os in Iraq 11 Py UNLoSkide to Ut Shipex, in Airp Im...\n6603     b'NG an Phangel Revelled    360 Ammi...\n8973                      b'Puecit EPDARDC'\n9695                       b'Eunta apenve ista erderterd pone'\n6042     b'EUrlurereakicy jeartad fur k ned thith oft costed 9...\n8690    b'AK Hegalan Qulical Sceltwor-te Pureyty Abaly'\n9984     b'Grown lead for talks  nonted blomb fromted bestack?'\n9355               b'New gnofit, Dallud for Shirm on Upp'\n9926    b'UR wind shaigh Eanouncon wititing of Echive Dea...\n9110    b'Cauma out

In [ ]:
one_step_model.save('NewsOneStep.keras')
custom_objects={'my_package':MyModel,'my_package2':OneStep}
with keras.saving.custom_object_scope(custom_objects=custom_objects):
  one_step_reloaded = tf.keras.models.load_model('NewsOneStep.keras',custom_objects=custom_objects)

states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

AttributeError: 'OneStep' object has no attribute 'save_model'